# COMP90086 Asmt 4 Model Main Control IPYNB

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# import sys
# import os
# py_file_location = "./drive/My Drive/LAB/COMP90086"
# sys.path.append(os.path.abspath(py_file_location))

# !pip install einops

### Load the libraries and network architecture

In [1]:
from environment import *
from utils import *
from model import *

### Load the data and pre-process the data

In [2]:
# ROOT = './drive/My Drive/LAB/COMP90086/COMP90086_2023_TLLdataset 2'
ROOT = './COMP90086_2023_TLLdataset 2'
seed = 19260817 # set seed
IMAGE_SIZE = 0
data_list = pd.read_csv(ROOT + '/train.csv')
future_list = pd.read_csv(ROOT + '/test_candidates.csv')

In [3]:
# preload all images

imgs = dict()
for img in tqdm(os.listdir(ROOT+'/train/left')):
    imgs[img] = prepare_image(ROOT+'/train/left/'+img, resize_shape=IMAGE_SIZE)

for img in tqdm(os.listdir(ROOT+'/train/right')):
    imgs[img] = prepare_image(ROOT+'/train/right/'+img, resize_shape=IMAGE_SIZE)

for img in tqdm(os.listdir(ROOT+'/test/left')):
    imgs[img] = prepare_image(ROOT+'/test/left/'+img, resize_shape=IMAGE_SIZE)

for img in tqdm(os.listdir(ROOT+'/test/right')):
    imgs[img] = prepare_image(ROOT+'/test/right/'+img, resize_shape=IMAGE_SIZE)

100%|██████████| 2000/2000 [00:01<00:00, 1374.15it/s]


In [4]:
# Train Val Test Split
train_list, valtest_list = train_test_split(data_list, test_size=0.3, random_state=seed) # train is 70%
val_list, test_list = train_test_split(valtest_list, test_size=0.5, random_state=seed) # val is 15%, test is 15%

# Siamese Contrastive

#### We have run Siamese Contrastive model with 4 different negative mining strategies and the result is shown below. The best result is achieved by using the semi-hard negative mining strategy (Experiment 2).

##### Experiment 1: No resampling

In [12]:
# Experiment 1: No Resampling

NAME = 'densenet201' # name of the CNN encoder on PyTorch

for _ in range(2):
    torch.cuda.empty_cache()
    gc.collect()

class CNN_Siamise_Triplet_CFG:
    num_false               = 9 # how many hard negative to try for each anchor
    encoder                 = NAME 
    freeze_encoder          = False # whether to freeze the pre-trained encoder weights
    input_shape             = (1, 3, IMAGE_SIZE, IMAGE_SIZE) if IMAGE_SIZE else (1, 3, 245, 200) # input image shape
    num_mlp_layers          = 2 # number of layers in MLP
    hidden_dim              = 512 # size of the hidden layer in MLP
    dropout                 = 0.1 # dropout rate for MLP
    res_learning            = False # whether to learn residual
    embed_dim               = 128 # size of the actual embedding for each image
    pretrained              = True # whether to use pre-trained weights
    crop_pretrained_linear  = True # whether to crop linear layer from imported CNN encoder
    final_relu              = False # whether to add ReLU to the embedding
    # ------------------------ #
    random_state            = seed 
    lr                      = 1e-5 # learning rate
    loss                    = nn.TripletMarginLoss(margin = 1) # loss function
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving
    # rootpath                = py_file_location
    DataFactory_Triplet     = DataFactory_Triplet # datafactory
    DataLoader_Triplet      = DataLoader_Triplet # dataloader
    images                  = imgs # loaded images
    name                    = NAME + '_siamese_triplet_Expr1' # name for saving model
    real_eval_batch_size    = 16 # real evaluation batch size
    resample                = False # whether to do non-random negative resampling
    num_random_sample_false = 0 # number of random negative to incorporate under non-random  negative sampling scheme
    semi_hard               = True # if using non-random negative resampling, whether to use semi-hard negative
    

cnn_siamise_triplet = CNN_Siamise_Triplet(CNN_Siamise_Triplet_CFG)
cnn_siamise_triplet.fit(train_list, val_list, batch_size = 16, epochs=100, patience=5, scheduler=True, grad_clip=False, mark='')

cnn_siamise_triplet.load()
print('Best Validation:')
cnn_siamise_triplet.eval(val_list, batch_size=16, return_loss=False)


print('Best Test:')
future_test_list = turn_val_into_future(test_list, seed)
cnn_siamise_triplet.real_eval(future_test_list, 1)

final_out1, score = cnn_siamise_triplet.real_eval(future_list, 1)
final_out1.to_csv(f'./{NAME}_siamese_triplet_Expr1.csv', index=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 1576/1576 [04:35<00:00,  5.72it/s]


Epoch:   1 - Loss: 0.167
Nominal Correct: 0.58
Validation Loss: 0.364


100%|██████████| 1576/1576 [04:33<00:00,  5.77it/s]


Epoch:   2 - Loss: 0.023
Nominal Correct: 0.5866666666666667
Validation Loss: 0.403


100%|██████████| 1576/1576 [04:32<00:00,  5.78it/s]


...

Epoch:   9 - Loss: 0.002
Nominal Correct: 0.6366666666666667
Validation Loss: 0.473


100%|██████████| 1576/1576 [04:34<00:00,  5.75it/s]


Epoch:  10 - Loss: 0.001
Nominal Correct: 0.63
Validation Loss: 0.453


100%|██████████| 1576/1576 [04:36<00:00,  5.70it/s]


Epoch:  11 - Loss: 0.001
Nominal Correct: 0.62
Validation Loss: 0.455
Best Validation:
Nominal Correct: 0.6566666666666666
Validation Loss: 0.458
Best Test:
Nominal Correct: 0.5533333333333333
Nominal Correct: 0.097


##### Experiment 2: Semi-hard negative mining

In [7]:
# Experiment 2: Semihard Resampling

NAME = 'densenet201'

for _ in range(2):
    torch.cuda.empty_cache()
    gc.collect()

class CNN_Siamise_Triplet_CFG:
    num_false               = 9 # how many hard negative to try for each anchor
    encoder                 = NAME
    freeze_encoder          = False
    input_shape             = (1, 3, IMAGE_SIZE, IMAGE_SIZE) if IMAGE_SIZE else (1, 3, 245, 200)
    num_mlp_layers          = 2
    hidden_dim              = 512
    dropout                 = 0.1
    res_learning            = False
    embed_dim               = 128 # size of the actual embedding for each image
    pretrained              = True      
    crop_pretrained_linear  = True
    final_relu              = False
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = nn.TripletMarginLoss(margin = 1) 
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving
    # rootpath                = py_file_location
    DataFactory_Triplet     = DataFactory_Triplet
    DataLoader_Triplet      = DataLoader_Triplet
    images                  = imgs
    name                    = NAME + '_siamese_triplet_Expr2'
    real_eval_batch_size    = 16
    resample                = True
    num_random_sample_false = 0
    semi_hard               = True
    

cnn_siamise_triplet = CNN_Siamise_Triplet(CNN_Siamise_Triplet_CFG)
cnn_siamise_triplet.fit(train_list, val_list, batch_size = 16, epochs=100, patience=5, scheduler=True, grad_clip=False, mark='')

cnn_siamise_triplet.load()
print('Best Validation:')
cnn_siamise_triplet.eval(val_list, batch_size=16, return_loss=False)


print('Best Test:')
future_test_list = turn_val_into_future(test_list, seed)
cnn_siamise_triplet.real_eval(future_test_list, 1)

final_out1, score = cnn_siamise_triplet.real_eval(future_list, 1)
final_out1.to_csv(f'./{NAME}_siamese_triplet_Expr2.csv', index=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 1576/1576 [04:34<00:00,  5.74it/s]


Epoch:   1 - Loss: 0.167
Nominal Correct: 0.56
Validation Loss: 0.395


100%|██████████| 1576/1576 [04:43<00:00,  5.57it/s]


Epoch:   2 - Loss: 0.129
Nominal Correct: 0.5433333333333333
Validation Loss: 0.414


100%|██████████| 1576/1576 [04:42<00:00,  5.58it/s]


...

Epoch:  10 - Loss: 0.001
Nominal Correct: 0.6933333333333334
Validation Loss: 0.388


100%|██████████| 1576/1576 [04:41<00:00,  5.60it/s]


Epoch:  11 - Loss: 0.001
Nominal Correct: 0.6933333333333334
Validation Loss: 0.402


100%|██████████| 1576/1576 [04:40<00:00,  5.61it/s]


Epoch:  12 - Loss: 0.002
Nominal Correct: 0.6633333333333333
Validation Loss: 0.411
Best Validation:
Nominal Correct: 0.6933333333333334
Validation Loss: 0.402
Best Test:
Nominal Correct: 0.6166666666666667
Nominal Correct: 0.096


##### Experiment 3: Hard negative mining

In [12]:
# Experiment 3: Hard Resampling
NAME = 'densenet201'

for _ in range(2):
    torch.cuda.empty_cache()
    gc.collect()

class CNN_Siamise_Triplet_CFG:
    num_false               = 9 # how many hard negative to try for each anchor
    encoder                 = NAME
    freeze_encoder          = False
    input_shape             = (1, 3, IMAGE_SIZE, IMAGE_SIZE) if IMAGE_SIZE else (1, 3, 245, 200)
    num_mlp_layers          = 2
    hidden_dim              = 512
    dropout                 = 0.1
    res_learning            = False
    embed_dim               = 128 # size of the actual embedding for each image
    pretrained              = True      
    crop_pretrained_linear  = True
    final_relu              = False
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = nn.TripletMarginLoss(margin = 1) 
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving
    # rootpath                = py_file_location
    DataFactory_Triplet     = DataFactory_Triplet
    DataLoader_Triplet      = DataLoader_Triplet
    images                  = imgs
    name                    = NAME + '_siamese_triplet_Expr3'
    real_eval_batch_size    = 16
    resample                = True
    num_random_sample_false = 0
    semi_hard               = False
    

cnn_siamise_triplet = CNN_Siamise_Triplet(CNN_Siamise_Triplet_CFG)
cnn_siamise_triplet.fit(train_list, val_list, batch_size = 16, epochs=100, patience=8, scheduler=True, grad_clip=False, mark='')

cnn_siamise_triplet.load()
print('Best Validation:')
cnn_siamise_triplet.eval(val_list, batch_size=16, return_loss=False)


print('Best Test:')
future_test_list = turn_val_into_future(test_list, seed)
cnn_siamise_triplet.real_eval(future_test_list, 1)

final_out1, score = cnn_siamise_triplet.real_eval(future_list, 1)
final_out1.to_csv(f'./{NAME}_siamese_triplet_Expr3.csv', index=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 1576/1576 [04:38<00:00,  5.66it/s]


Epoch:   1 - Loss: 0.167
Nominal Correct: 0.56
Validation Loss: 0.367


100%|██████████| 1576/1576 [04:45<00:00,  5.52it/s]


Epoch:   2 - Loss: 0.222
Nominal Correct: 0.5666666666666667
Validation Loss: 0.437


100%|██████████| 1576/1576 [04:47<00:00,  5.49it/s]


...

Epoch:  14 - Loss: 0.112
Nominal Correct: 0.6533333333333333
Validation Loss: 0.457


100%|██████████| 1576/1576 [04:43<00:00,  5.56it/s]


Epoch:  15 - Loss: 0.112
Nominal Correct: 0.66
Validation Loss: 0.469


100%|██████████| 1576/1576 [04:46<00:00,  5.50it/s]


Epoch:  16 - Loss: 0.112
Nominal Correct: 0.6666666666666666
Validation Loss: 0.479
Best Validation:
Nominal Correct: 0.6933333333333334
Validation Loss: 0.445
Best Test:
Nominal Correct: 0.5933333333333334
Nominal Correct: 0.097


##### Experiment 4: Semi-hard negative mining with random negative sampling

In [9]:
# Experiment 4: Semihard Resampling + Random
NAME = 'densenet201'

for _ in range(2):
    torch.cuda.empty_cache()
    gc.collect()

class CNN_Siamise_Triplet_CFG:
    num_false               = 9 # how many hard negative to try for each anchor
    encoder                 = NAME
    freeze_encoder          = False
    input_shape             = (1, 3, IMAGE_SIZE, IMAGE_SIZE) if IMAGE_SIZE else (1, 3, 245, 200)
    num_mlp_layers          = 2
    hidden_dim              = 512
    dropout                 = 0.1
    res_learning            = False
    embed_dim               = 128 # size of the actual embedding for each image
    pretrained              = True      
    crop_pretrained_linear  = True
    final_relu              = False
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = nn.TripletMarginLoss(margin = 1) 
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving
    # rootpath                = py_file_location
    DataFactory_Triplet     = DataFactory_Triplet
    DataLoader_Triplet      = DataLoader_Triplet
    images                  = imgs
    name                    = NAME + '_siamese_triplet_Expr4'
    real_eval_batch_size    = 16
    resample                = True
    num_random_sample_false = 4
    semi_hard               = True
    

cnn_siamise_triplet = CNN_Siamise_Triplet(CNN_Siamise_Triplet_CFG)
cnn_siamise_triplet.fit(train_list, val_list, batch_size = 16, epochs=100, patience=8, scheduler=True, grad_clip=False, mark='')

cnn_siamise_triplet.load()
print('Best Validation:')
cnn_siamise_triplet.eval(val_list, batch_size=16, return_loss=False)


print('Best Test:')
future_test_list = turn_val_into_future(test_list, seed)
cnn_siamise_triplet.real_eval(future_test_list, 1)

final_out1, score = cnn_siamise_triplet.real_eval(future_list, 1)
final_out1.to_csv(f'./{NAME}_siamese_triplet_Expr4.csv', index=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 1576/1576 [04:55<00:00,  5.34it/s]


Epoch:   1 - Loss: 0.168
Nominal Correct: 0.5866666666666667
Validation Loss: 0.374


100%|██████████| 1576/1576 [04:50<00:00,  5.43it/s]


Epoch:   2 - Loss: 0.117
Nominal Correct: 0.5933333333333334
Validation Loss: 0.372


100%|██████████| 1576/1576 [04:55<00:00,  5.34it/s]


...

Epoch:  18 - Loss: 0.001
Nominal Correct: 0.6866666666666666
Validation Loss: 0.417


100%|██████████| 1576/1576 [04:49<00:00,  5.45it/s]


Epoch:  19 - Loss: 0.000
Nominal Correct: 0.71
Validation Loss: 0.417


100%|██████████| 1576/1576 [04:53<00:00,  5.37it/s]


Epoch:  20 - Loss: 0.000
Nominal Correct: 0.6933333333333334
Validation Loss: 0.418
Best Validation:
Nominal Correct: 0.71
Validation Loss: 0.417
Best Test:
Nominal Correct: 0.5533333333333333
Nominal Correct: 0.0885


In [8]:
# Experiment 5: Double Triplet Loss
NAME = 'densenet201'

for _ in range(2):
    torch.cuda.empty_cache()
    gc.collect()

class CNN_Siamise_Triplet_CFG:
    num_false               = 9 # how many hard negative to try for each anchor
    encoder                 = NAME
    freeze_encoder          = False
    input_shape             = (1, 3, IMAGE_SIZE, IMAGE_SIZE) if IMAGE_SIZE else (1, 3, 245, 200)
    num_mlp_layers          = 2
    hidden_dim              = 512
    dropout                 = 0.1
    res_learning            = False
    embed_dim               = 128 # size of the actual embedding for each image
    pretrained              = True      
    crop_pretrained_linear  = True
    final_relu              = False
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = Double_Triplet_Loss(margin=1.0, margin_p = 0.5)
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving
    # rootpath                = py_file_location
    DataFactory_Triplet     = DataFactory_Triplet
    DataLoader_Triplet      = DataLoader_Triplet
    images                  = imgs
    name                    = NAME + '_siamese_triplet_Expr5'
    real_eval_batch_size    = 16
    resample                = True
    num_random_sample_false = 0
    semi_hard               = True
    

cnn_siamise_triplet = CNN_Siamise_Triplet(CNN_Siamise_Triplet_CFG)
cnn_siamise_triplet.fit(train_list, val_list, batch_size = 16, epochs=100, patience=8, scheduler=True, grad_clip=False, mark='')

cnn_siamise_triplet.load()
print('Best Validation:')
cnn_siamise_triplet.eval(val_list, batch_size=16, return_loss=False)


print('Best Test:')
future_test_list = turn_val_into_future(test_list, seed)
cnn_siamise_triplet.real_eval(future_test_list, 1)

final_out1, score = cnn_siamise_triplet.real_eval(future_list, 1)
final_out1.to_csv(f'./{NAME}_siamese_triplet_Expr5.csv', index=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 1576/1576 [05:00<00:00,  5.25it/s]


Epoch:   1 - Loss: 0.270
Nominal Correct: 0.5933333333333334
Validation Loss: 1.054


100%|██████████| 1576/1576 [04:51<00:00,  5.40it/s]


Epoch:   2 - Loss: 0.135
Nominal Correct: 0.5866666666666667
Validation Loss: 1.115


100%|██████████| 1576/1576 [04:51<00:00,  5.41it/s]


...

Epoch:  10 - Loss: 0.023
Nominal Correct: 0.64
Validation Loss: 1.243


100%|██████████| 1576/1576 [04:53<00:00,  5.38it/s]


Epoch:  11 - Loss: 0.027
Nominal Correct: 0.6266666666666667
Validation Loss: 1.304


100%|██████████| 1576/1576 [04:49<00:00,  5.44it/s]


Epoch:  12 - Loss: 0.004
Nominal Correct: 0.62
Validation Loss: 1.327
Best Validation:
Nominal Correct: 0.6433333333333333
Validation Loss: 1.118
Best Test:
Nominal Correct: 0.5733333333333334
Nominal Correct: 0.094


# Key Query Contrastive

In [ ]:
NAME = 'densenet201'

for _ in range(2):
    torch.cuda.empty_cache()
    gc.collect()
    
class CNN_Key_Query_Triplet_CFG:
    num_false               = 9 # how many hard negative to try for each anchor
    encoder                 = NAME
    freeze_encoder          = False
    input_shape             = (64, 3, IMAGE_SIZE, IMAGE_SIZE) if IMAGE_SIZE else (64, 3, 245, 200)
    num_mlp_layers          = 2
    hidden_dim              = 512
    dropout                 = 0.1
    res_learning            = False
    embed_dim               = 128 # size of the actual embedding for each image
    pretrained              = True      
    crop_pretrained_linear  = True
    final_relu              = False
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = nn.TripletMarginLoss() 
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving
    # rootpath                = py_file_location
    DataFactory_Triplet     = DataFactory_Triplet
    DataLoader_Triplet      = DataLoader_Triplet
    images                  = imgs
    name                    = NAME + 'key_query'
    real_eval_batch_size    = 16
    resample                = True
    num_random_sample_false = 0
    semi_hard               = True
    

cnn_key_query_triplet = CNN_Key_Query_Triplet(CNN_Key_Query_Triplet_CFG)
cnn_key_query_triplet.fit(train_list, val_list, batch_size = 16, epochs=100, patience=5, scheduler=True, grad_clip=False, mark='')

cnn_key_query_triplet.load()
print('Best Validation:')
cnn_key_query_triplet.eval(val_list, batch_size=16, return_loss=False)


print('Best Test:')
future_test_list = turn_val_into_future(test_list, seed)
cnn_key_query_triplet.real_eval(future_test_list, 2)

final_out1, score = cnn_key_query_triplet.real_eval(future_list, 2)
final_out1.to_csv(f'./{NAME}_key_query_triplet.csv', index=False)

In [ ]:
# Training Record

# 100%|██████████| 1576/1576 [05:01<00:00,  5.23it/s]
# Epoch:   1 - Loss: 0.308
# Nominal Correct: 0.5233333333333333
# Validation Loss: 0.396
# 100%|██████████| 1576/1576 [05:05<00:00,  5.16it/s]
# Epoch:   2 - Loss: 0.366
# Nominal Correct: 0.44
# Validation Loss: 0.538
# 100%|██████████| 1576/1576 [05:02<00:00,  5.21it/s]
# Epoch:   3 - Loss: 0.125
# Nominal Correct: 0.55
# Validation Loss: 0.401
# 100%|██████████| 1576/1576 [05:02<00:00,  5.21it/s]
# Epoch:   4 - Loss: 0.096
# Nominal Correct: 0.49666666666666665
# Validation Loss: 0.527
# 100%|██████████| 1576/1576 [05:07<00:00,  5.12it/s]
# Epoch:   5 - Loss: 0.031
# Nominal Correct: 0.6133333333333333
# Validation Loss: 0.386
# 100%|██████████| 1576/1576 [05:08<00:00,  5.11it/s]
# Epoch:   6 - Loss: 0.029
# Nominal Correct: 0.5666666666666667
# Validation Loss: 0.438
# 100%|██████████| 1576/1576 [05:11<00:00,  5.05it/s]
# Epoch:   7 - Loss: 0.016
# Nominal Correct: 0.5966666666666667
# Validation Loss: 0.406
# 100%|██████████| 1576/1576 [05:08<00:00,  5.12it/s]
# Epoch:   8 - Loss: 0.015
# Nominal Correct: 0.6166666666666667
# Validation Loss: 0.394
# 100%|██████████| 1576/1576 [05:09<00:00,  5.10it/s]
# Epoch:   9 - Loss: 0.004
# Nominal Correct: 0.64
# Validation Loss: 0.391
# 100%|██████████| 1576/1576 [05:05<00:00,  5.17it/s]
# Epoch:  10 - Loss: 0.003
# Nominal Correct: 0.6533333333333333
# Validation Loss: 0.393
# 100%|██████████| 1576/1576 [05:06<00:00,  5.14it/s]
# Epoch:  11 - Loss: 0.004
# Nominal Correct: 0.5866666666666667
# Validation Loss: 0.429

# Best Validation:
# Nominal Correct: 0.6533333333333333
# Validation Loss: 0.393
# Best Test:
# Nominal Correct: 0.57
# Nominal Correct: 0.0925

# Siamise Categorisation

In [ ]:
NAME = 'densenet121'

for _ in range(2):
    torch.cuda.empty_cache()
    gc.collect()

class CNN_Siamise_Categorisation_CFG:
    num_false               = 9 # how many hard negative to try for each anchor
    encoder                 = NAME
    freeze_encoder          = False
    input_shape             = (1, 3, IMAGE_SIZE, IMAGE_SIZE) if IMAGE_SIZE else (1, 3, 245, 200)
    num_mlp_layers          = 2
    hidden_dim              = 512
    dropout                 = 0.1
    res_learning            = False
    embed_dim               = 128 # size of the actual embedding for each image
    pretrained              = True      
    crop_pretrained_linear  = True
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = nn.BCELoss() 
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving
    # rootpath                = py_file_location
    DataFactory_Triplet     = DataFactory_Triplet
    DataLoader_Triplet      = DataLoader_Triplet
    images                  = imgs
    name                    = NAME +'siamese_categorisation'
    real_eval_batch_size    = 10

cnn_siamise_categorisation = CNN_Siamise_Categorisation(CNN_Siamise_Categorisation_CFG)
cnn_siamise_categorisation.fit(train_list, val_list, batch_size = 32, epochs=100, patience=8, scheduler=True, grad_clip=False, mark='')

cnn_siamise_categorisation.load()
print('Best Validation:')
cnn_siamise_categorisation.eval(val_list, batch_size=32, return_loss=False)


print('Best Test:')
future_test_list = turn_val_into_future(test_list, seed)
cnn_siamise_categorisation.real_eval(future_test_list, 10)

final_out2 = cnn_siamise_categorisation.real_eval(future_list, 10)
final_out2.to_csv(f'./{NAME}_siamese_categorisation.csv', index=False)

# Key Query Categorisation

In [ ]:
NAME = 'resnet'

for _ in range(2):
    torch.cuda.empty_cache()
    gc.collect()

class CNN_Key_Query_Categorisation_CFG:
    num_false               = 1 # how many false samples for each true sample
    target                  = 1 # target loss for correct. false will be 1-target
    encoder                 = NAME # the name of the model we want to load from torchhub
    freeze_encoder          = True
    input_shape             = (64, 3, IMAGE_SIZE, IMAGE_SIZE) if IMAGE_SIZE else (64, 3, 245, 200)  # (batch size, 3, [what our loaded in images look like])
    num_mlp_layers          = 2 # how many mlp layers in the end. 0 means no mlp and just 1 out layer; n means n mlp + outlayer
    hidden_dim              = 512 # hidden dim of mlp
    dropout                 = 0.1 # dropout rate of mlp
    res_learning            = False # whether to use residual layer in mlp
    pretrained              = True # whether to use pretrained encoder
    crop_pretrained_linear  = True
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = nn.BCELoss()   
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpatbbh for saving models
    # rootpath                = py_file_location
    DataFactory_Categorisation = DataFactory_Categorisation
    DataLoader_Categorisation = DataLoader_Categorisation
    images                  = imgs
    name                    = NAME + '_categorisation'
    real_eval_batch_size    = 64
    resample                = True
    num_random_sample       = 9


cnn_key_query_categorisation = CNN_Key_Query_Categorisation(CNN_Key_Query_Categorisation_CFG)
future_test_list = turn_val_into_future(test_list, seed)
cnn_key_query_categorisation.real_eval(future_test_list, 21)
cnn_key_query_categorisation.fit(train_list, val_list, batch_size = 64, epochs=100, patience=8, scheduler=True, grad_clip=False, mark='')

cnn_key_query_categorisation.load()
print('Best Validation:')
cnn_key_query_categorisation.eval(val_list, batch_size=128, return_loss=False)

print('Best Test:')
future_test_list = turn_val_into_future(test_list, seed)
cnn_key_query_categorisation.real_eval(future_test_list, 100)

final_out2 = cnn_key_query_categorisation.real_eval(future_list, 100)
final_out2.to_csv(f'./{NAME}_key_query_categorisation.csv', index=False)